In [4]:
import pandas as pd
from random import randint

In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/37/zw9ym1bn7f31rcp5g2cvjr8m0000gn/T/ipykernel_3109/3871533204.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [13]:
from selenium.webdriver.common.by import By
from dateutil import parser

symbols = ['STAA', 'STTP', 'ROTI', 'LSIP', 'TLDN', 'SIMP', 'PALM']

from time import sleep

df = pd.DataFrame()

for symbol in symbols:
    driver.get(f"https://stockbit.com/symbol/{symbol}")
    
    # Handdle: IndexError: list index out of range
    while True:
        date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        
        # Handle: IndexError: list index out of range
        if len(date_elements) != 0:
            break

    while True:
        # date_elements[-2] Handle: StaleElementReferenceException or ''
        # .text[:16] Handle: ParserError: Unknown string format: 14 Dec 22, 19:42 · Edited on 14 Dec 22, 21:29
        if parser.parse(date_elements[-2].text[:16]) >= parser.parse("1 Oct 22"):
            driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
            date_elements = driver.find_elements(by=By.XPATH, value="//div[@class='stream-date']")
        else:
            break
    
    # value='stream-text-single' Handle: prevent Repost to be included in the comment_elements    
    comment_elements = driver.find_elements(by=By.CLASS_NAME, value="stream-text-single")
    
    dates = list()
    comments = list()
    
    for index, (date_element, comment_element) in enumerate(zip(date_elements, comment_elements)):
        # print(date_element.text, comment_element.text, "\n")
        
        # Handle: StaleElementReferenceException or ''
        if index == len(date_elements) - 1:
            break
        
        dates.append(date_element.text)
        comments.append(comment_element.text)
        
    print(f'symbol {symbol} '
          f'date_elements {len(date_elements)} dates {len(dates)} '
          f'comment_elements {len(comment_elements)} comments {len(comments)} ')
    
    df_c = pd.DataFrame()
    df_c['date'] = dates
    df_c['comment'] = comments
    df_c['symbol'] = symbol
    
    df = pd.concat([df, df_c])
    
    sleep(randint(5,10))

symbol STAA date_elements 641 dates 640 comment_elements 641 comments 640 
symbol STTP date_elements 41 dates 40 comment_elements 41 comments 40 
symbol ROTI date_elements 80 dates 79 comment_elements 80 comments 79 
symbol LSIP date_elements 361 dates 360 comment_elements 361 comments 360 
symbol TLDN date_elements 41 dates 40 comment_elements 41 comments 40 
symbol SIMP date_elements 141 dates 140 comment_elements 141 comments 140 
symbol PALM date_elements 121 dates 120 comment_elements 121 comments 120 


In [15]:
df

,date,comment,symbol
0,"16 Dec 22, 13:19",$STAA selama masih ada potensi positif di haka...,STAA
1,"16 Dec 22, 11:21",$LSIP $BWPT $SSMS $STAA B35 apa kabar $BOSS ?g...,STAA
2,"16 Dec 22, 08:38",Harga kontrak Crude Palm Oil (CPO) di Bursa Ma...,STAA
3,"15 Dec 22, 08:17",$STAA membuka peluang membentuk tren naik sete...,STAA
4,"14 Dec 22, 22:53","JAKARTA, https://cutt.ly/H0xp1zp - Harga kontr...",STAA
...,...,...,...
115,"28 Sep 22, 06:33 · Edited on 28 Sep 22, 07:52",🎖 Pantauan Saham🎖\n28-09-22\n\n✅ $DOID\n✅ $OPM...,PALM
116,"27 Sep 22, 12:07 · Edited on 3 Oct 22, 17:36",$GZCO aja😁😁😁\n$ELSA\n$MEDC\n$AKRA\n$PALM,PALM
117,"27 Sep 22, 11:00 · Edited on 27 Sep 22, 11:53",$GZCO\n$ELSA\n$MEDC\n$PALM\n$AKRA\n\n\nKemarin...,PALM
118,"26 Sep 22, 07:45",$PALM,PALM


In [14]:
df.to_csv("STAA_PALM.csv", index=False)